In [1]:

# imports
import os
import sys
import types
import json
import base64

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96
interactivity = ''
is_shiny = False
is_dashboard = False
plotly_connected = True

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = "figure"

  # IPython 7.14 deprecated set_matplotlib_formats from IPython
  try:
    from matplotlib_inline.backend_inline import set_matplotlib_formats
  except ImportError:
    # Fall back to deprecated location for older IPython versions
    from IPython.display import set_matplotlib_formats
    
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  if plotly_connected:
    pio.renderers.default = "notebook_connected"
  else:
    pio.renderers.default = "notebook"
  for template in pio.templates.keys():
    pio.templates[template].layout.margin = dict(t=30,r=0,b=0,l=0)
except Exception:
  pass

# disable itables paging for dashboards
if is_dashboard:
  try:
    from itables import options
    options.dom = 'fiBrtlp'
    options.maxBytes = 1024 * 1024
    options.language = dict(info = "Showing _TOTAL_ entries")
    options.classes = "display nowrap compact"
    options.paging = False
    options.searching = True
    options.ordering = True
    options.info = True
    options.lengthChange = False
    options.autoWidth = False
    options.responsive = True
    options.keys = True
    options.buttons = []
  except Exception:
    pass
  
  try:
    import altair as alt
    # By default, dashboards will have container sized
    # vega visualizations which allows them to flow reasonably
    theme_sentinel = '_quarto-dashboard-internal'
    def make_theme(name):
        nonTheme = alt.themes._plugins[name]    
        def patch_theme(*args, **kwargs):
            existingTheme = nonTheme()
            if 'height' not in existingTheme:
              existingTheme['height'] = 'container'
            if 'width' not in existingTheme:
              existingTheme['width'] = 'container'

            if 'config' not in existingTheme:
              existingTheme['config'] = dict()
            
            # Configure the default font sizes
            title_font_size = 15
            header_font_size = 13
            axis_font_size = 12
            legend_font_size = 12
            mark_font_size = 12
            tooltip = False

            config = existingTheme['config']

            # The Axis
            if 'axis' not in config:
              config['axis'] = dict()
            axis = config['axis']
            if 'labelFontSize' not in axis:
              axis['labelFontSize'] = axis_font_size
            if 'titleFontSize' not in axis:
              axis['titleFontSize'] = axis_font_size  

            # The legend
            if 'legend' not in config:
              config['legend'] = dict()
            legend = config['legend']
            if 'labelFontSize' not in legend:
              legend['labelFontSize'] = legend_font_size
            if 'titleFontSize' not in legend:
              legend['titleFontSize'] = legend_font_size  

            # The header
            if 'header' not in config:
              config['header'] = dict()
            header = config['header']
            if 'labelFontSize' not in header:
              header['labelFontSize'] = header_font_size
            if 'titleFontSize' not in header:
              header['titleFontSize'] = header_font_size    

            # Title
            if 'title' not in config:
              config['title'] = dict()
            title = config['title']
            if 'fontSize' not in title:
              title['fontSize'] = title_font_size

            # Marks
            if 'mark' not in config:
              config['mark'] = dict()
            mark = config['mark']
            if 'fontSize' not in mark:
              mark['fontSize'] = mark_font_size

            # Mark tooltips
            if tooltip and 'tooltip' not in mark:
              mark['tooltip'] = dict(content="encoding")

            return existingTheme
            
        return patch_theme

    # We can only do this once per session
    if theme_sentinel not in alt.themes.names():
      for name in alt.themes.names():
        alt.themes.register(name, make_theme(name))
      
      # register a sentinel theme so we only do this once
      alt.themes.register(theme_sentinel, make_theme('default'))
      alt.themes.enable('default')

  except Exception:
    pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass

# interactivity
if interactivity:
  from IPython.core.interactiveshell import InteractiveShell
  InteractiveShell.ast_node_interactivity = interactivity

# NOTE: the kernel_deps code is repeated in the cleanup.py file
# (we can't easily share this code b/c of the way it is run).
# If you edit this code also edit the same code in cleanup.py!

# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
run_path = 'RDpcQ2xvdWRcU2l0ZXNfSW50ZXJuZXRccXVhcnRvX1BOU0lcMjJfdGFibGVz'
if run_path:
  # hex-decode the path
  run_path = base64.b64decode(run_path.encode("utf-8")).decode("utf-8")
  os.chdir(run_path)

# reset state
%reset

# shiny
# Checking for shiny by using False directly because we're after the %reset. We don't want
# to set a variable that stays in global scope.
if False:
  try:
    import htmltools as _htmltools
    import ast as _ast

    _htmltools.html_dependency_render_mode = "json"

    # This decorator will be added to all function definitions
    def _display_if_has_repr_html(x):
      try:
        # IPython 7.14 preferred import
        from IPython.display import display, HTML
      except:
        from IPython.core.display import display, HTML

      if hasattr(x, '_repr_html_'):
        display(HTML(x._repr_html_()))
      return x

    # ideally we would undo the call to ast_transformers.append
    # at the end of this block whenver an error occurs, we do 
    # this for now as it will only be a problem if the user 
    # switches from shiny to not-shiny mode (and even then likely
    # won't matter)
    import builtins
    builtins._display_if_has_repr_html = _display_if_has_repr_html

    class _FunctionDefReprHtml(_ast.NodeTransformer):
      def visit_FunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

      def visit_AsyncFunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

    ip = get_ipython()
    ip.ast_transformers.append(_FunctionDefReprHtml())

  except:
    pass

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v

  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define
globals()["__spec__"] = None

{"C:\\Python312\\Lib\\importlib\\_bootstrap.py": 1725651074.0, "C:\\Python312\\Lib\\importlib\\_bootstrap_external.py": 1725651074.0, "C:\\Python312\\Lib\\zipimport.py": 1725651074.0, "C:\\Python312\\Lib\\codecs.py": 1725651074.0, "C:\\Python312\\Lib\\encodings\\aliases.py": 1725651074.0, "C:\\Python312\\Lib\\encodings\\__init__.py": 1725651074.0, "C:\\Python312\\Lib\\encodings\\utf_8.py": 1725651074.0, "C:\\Python312\\Lib\\encodings\\cp1252.py": 1725651074.0, "C:\\Python312\\Lib\\abc.py": 1725651074.0, "C:\\Python312\\Lib\\io.py": 1725651074.0, "C:\\Python312\\Lib\\stat.py": 1725651074.0, "C:\\Python312\\Lib\\_collections_abc.py": 1725651074.0, "C:\\Python312\\Lib\\genericpath.py": 1725651074.0, "C:\\Python312\\Lib\\ntpath.py": 1725651074.0, "C:\\Python312\\Lib\\os.py": 1725651074.0, "C:\\Python312\\Lib\\_sitebuiltins.py": 1725651074.0, "C:\\Python312\\Lib\\encodings\\utf_8_sig.py": 1725651074.0, "C:\\Python312\\Lib\\site-packages\\_distutils_hack\\__init__.py": 1732804422.8847911, "C

In [2]:
# ouverture du fichier en mode lecture
fichier = open('educ_cantal.csv', mode='r', encoding='utf8', newline='')
# extraction de la ligne des attributs
attributs = fichier.readline().rstrip().split(',')
# extraction des autres lignes
table = [ligne.rstrip().split(',') for ligne in fichier]
# fermeture du fichier
fichier.close()

In [3]:
print(attributs)
print(table[18])
print(table[:2])

['code', 'nom', 'statut', 'codepostal', 'id_commune', 'latitude', 'longitude']
['0150030B', 'Lycée polyvalent de Haute Auvergne', 'Public', '15101', '187', '45.0352036275507', '3.0819798437208243']
[['0150001V', 'Collège Maurice Peschaud', 'Public', '15160', '1', '45.230605740493495', '2.9353501250960568'], ['0150005Z', 'Collège La Jordanne', 'Public', '15005', '14', '44.92112535663867', '2.4404905355487023']]


In [4]:
g = open('educ_cantal_copie.csv', mode='w', encoding='utf8', newline='') # ouverture du fichier en mode écriture
premiere_ligne = ','.join(attributs) + '\n'
g.write(premiere_ligne)
for enregistrement in table:
    g.write(','.join(enregistrement) + '\n')
g.close()

In [5]:
import csv

f = open('educ_cantal.csv', mode='r', encoding='utf8', newline='')
reader = csv.DictReader(f, delimiter=',')  # création d'un objet reader
table = [enregistrement for enregistrement in reader]
f.close()

In [6]:
# premier enregistrement de type dictionnaire
print(table[18])
# tableu de dictionnaires : extrait
print(table[:2])
# nombre d'enregistrements
print(len(table))
# accès facile aux valeurs des attributs par clefs
print(table[18]["nom"])

{'code': '0150030B', 'nom': 'Lycée polyvalent de Haute Auvergne', 'statut': 'Public', 'codepostal': '15101', 'id_commune': '187', 'latitude': '45.0352036275507', 'longitude': '3.0819798437208243'}
[{'code': '0150001V', 'nom': 'Collège Maurice Peschaud', 'statut': 'Public', 'codepostal': '15160', 'id_commune': '1', 'latitude': '45.230605740493495', 'longitude': '2.9353501250960568'}, {'code': '0150005Z', 'nom': 'Collège La Jordanne', 'statut': 'Public', 'codepostal': '15005', 'id_commune': '14', 'latitude': '44.92112535663867', 'longitude': '2.4404905355487023'}]
203
Lycée polyvalent de Haute Auvergne


In [7]:
g = open('educ_cantal_copie2.csv', mode='w', encoding='utf8', newline='')
attributs = list(table[0].keys())
writer = csv.DictWriter(g, delimiter=',', fieldnames=attributs)  # création de l'objet writer
writer.writeheader()  # écriture des attributs
for enregistrement in table:
    writer.writerow(enregistrement)  # écriture des enregistrements
g.close()

In [8]:
f = open('educ_cantal.csv', mode='r', encoding='utf8', newline='')
reader = csv.reader(f, delimiter=',')
table = [ligne for ligne in reader]
f.close()

In [9]:
Table1 = [
    {'Nom': 'Joe', 'Anglais': '17', 'Info': '18', 'Maths': '16'},
    {'Nom': 'Zoé', 'Anglais': '15', 'Info': '17', 'Maths': '19'},
    {'Nom': 'Max', 'Anglais': '19', 'Info': '13', 'Maths': '14'},
    {'Nom': 'Bob', 'Anglais': '12', 'Info': '16', 'Maths': '10'}
]

In [10]:
au_moins_16_en_maths = []
for enregistrement in Table1:
    if int(enregistrement['Maths']) >= 16:
        au_moins_16_en_maths.append(enregistrement)

In [11]:
au_moins_16_en_maths

[{'Nom': 'Joe', 'Anglais': '17', 'Info': '18', 'Maths': '16'},
 {'Nom': 'Zoé', 'Anglais': '15', 'Info': '17', 'Maths': '19'}]

In [12]:
au_moins_16_en_maths = [enre for enre in Table1 if int(enre['Maths']) >= 16]

In [13]:
notes_maths = []
for enregistrement in Table1:
    notes_maths.append(enregistrement['Maths'])

In [14]:
notes_maths = [enre['Maths'] for enre in Table1]

In [15]:
nom_avec_o = []
for enregistrement in Table1:
    if 'o' in enregistrement['Nom']:
        nom_avec_o.append(enregistrement)

In [16]:
nom_avec_o = [enre for enre in Table1 if 'o' in enre['Nom']]

In [17]:
# avec une boucle
liste_noms = []
for enregistrement in Table1:
    liste_noms.append(enregistrement['Nom'])

# avec une liste en compréhension
liste_noms = [enre['Nom'] for enre in Table1]

In [18]:
# avec une boucle
liste_paires = []
for enregistrement in Table1:
    liste_paires.append((enregistrement['Info'], enregistrement['Maths']))

# avec une liste en compréhension
liste_paires = [(enre['Info'], enre['Maths']) for enre in Table1]

In [19]:
ma_liste = [10, 3, 71, 96]
print(sorted(ma_liste))
print(sorted(ma_liste, reverse=True))

mes_couples = [('a', 3), ('d', 2), ('c', 5), ('b', 1)]
print(sorted(mes_couples))     # tuples triés selon leur premier élément (par défaut)

def y(couple):    # on définit une fonction qui retourne le second élément d'un tuple
    return couple[1]

print(sorted(mes_couples, key=y)) # tuples triés selon leur second élément

[3, 10, 71, 96]
[96, 71, 10, 3]
[('a', 3), ('b', 1), ('c', 5), ('d', 2)]
[('b', 1), ('d', 2), ('a', 3), ('c', 5)]


In [20]:
table = [{'élève' : 'guido', 'langage' : 'python', 'note' : 19},
         {'élève' : 'monty', 'langage' : 'python', 'note' : 20},
         {'élève' : 'brian', 'langage' : 'c', 'note' : 20}]

In [21]:
def cle_langage_note(enre):
    return (enre['langage'], enre['note'])

sorted(table, key = cle_langage_note)

[{'élève': 'brian', 'langage': 'c', 'note': 20},
 {'élève': 'guido', 'langage': 'python', 'note': 19},
 {'élève': 'monty', 'langage': 'python', 'note': 20}]

In [22]:
def cle_langage(enre):
    return enre['langage']

def cle_note(enre):
    return enre['note']

table_temp = sorted(table, key = cle_langage)
sorted(table_temp, key = cle_note, reverse = True)

[{'élève': 'brian', 'langage': 'c', 'note': 20},
 {'élève': 'monty', 'langage': 'python', 'note': 20},
 {'élève': 'guido', 'langage': 'python', 'note': 19}]

In [23]:
def fusion_tables(table1: list, table2: list) -> list:
    """table 1 et table 2 ont les mêmes descripteurs
    il n'y a pas de doublons"""
    fusion = deepcopy(table1)
    for enreg in table2[1:]:    # on élimine la première ligne (descripteurs)
        fusion.append(deepcopy(enreg))
    return fusion

In [24]:
from copy import deepcopy

Table1 = [{'Nom': 'Joe', 'Anglais': '17', 'Info': '18', 'Maths': '16'},
          {'Nom': 'Zoé', 'Anglais': '15', 'Info': '17', 'Maths': '19'},
          {'Nom': 'Max', 'Anglais': '19', 'Info': '13', 'Maths': '14'},
          {'Nom': 'Bob', 'Anglais': '12', 'Info': '16', 'Maths': '10'}]

Table2 = [{'Nom': 'Joe', 'Age': 16, 'Couriel': 'joe@info.fr'},
          {'Nom': 'Zoé', 'Age': 15, 'Couriel': 'zoe@info.fr'}]


def jointure_selon_attribut_commun(t1: list, t2: list, attr: str) -> list:
    """Effectue la jointure entre les tables t1 et t2 selon l'attribut commun attr"""
    jointure = []
    for item1 in t1:
        for item2 in t2:
            if item1[attr] == item2[attr]:
                # définition d'une ligne de la nouvelle table reprenant le contenu de la ligne en cours de t1
                new_line = deepcopy(item1)
                # boucle pour ajouter les champs de la ligne de t2 autres que attr
                for cle in item2:
                    if cle != attr:
                        new_line[cle] = item2[cle]
                # ajout de la nouvelle ligne à la nouvelle table
                jointure.append(new_line)
    return jointure


print(jointure_selon_attribut_commun(Table2, Table1, 'Nom'))

[{'Nom': 'Joe', 'Age': 16, 'Couriel': 'joe@info.fr', 'Anglais': '17', 'Info': '18', 'Maths': '16'}, {'Nom': 'Zoé', 'Age': 15, 'Couriel': 'zoe@info.fr', 'Anglais': '15', 'Info': '17', 'Maths': '19'}]
